In [20]:
!pip install langchain
!pip install gigachat
!pip install -U gigachain_core

In [21]:
import pandas as pd

from langchain.prompts import load_prompt
from langchain.chat_models import GigaChat
from langchain.chains import LLMChain

In [22]:
def gigachat_function(creds, df_path, prompt_path, filename):
  # reads df from path
  df = pd.read_csv(df_path, sep='\t')

  #calls gigachat with unique credentials
  giga = GigaChat(
    credentials=creds,
    scope='GIGACHAT_API_PERS', verify_ssl_certs=False)
  prompt = load_prompt(prompt_path)
  chain = prompt | giga

  #reads all words and corresponding sentences
  target_dict = []
  for i in range(len(df['word'].tolist())):
    target_dict.append({df['word'].tolist()[i]: [df['sent1'].tolist()[i], df['sent2'].tolist()[i]]})

  #process data with gigachat according to prompt
  grade_dict = []
  for i in range(len(target_dict)):
    giga_grade = chain.invoke({"word": list(target_dict[i].items())[0][0],
                        "text1": list(target_dict[i].items())[0][1][0],
                        "text2": list(target_dict[i].items())[0][1][1]}).content
    grade_dict.append({list(target_dict[i].items())[0][0]: giga_grade})
  print(grade_dict)

  #accumulate and calculate means o gigachat predictions
  accumulate_shift = {i: [] for i in sorted(list(set(df['word'].tolist())))}
  for smth in range(len(grade_dict)):
    try:
      accumulate_shift[str(list(grade_dict[smth].keys())[0])].append(int(list(grade_dict[smth].values())[0]))
    except Exception as e:
      continue
  res_dict = {}
  for k, v in accumulate_shift.items():
    res_dict.update({k: sum([int(i) for i in v])/len([int(i) for i in v])})
  df_dict = {'word': list(res_dict.keys()), 'mean': list(res_dict.values())}
  result = pd.DataFrame.from_dict(df_dict)
  result.to_csv(filename, header=False, index=False)

In [10]:
gigachat_function('MTgzMjI0NTktMDA3Ny00Y2NlLTg4MDctMzkwNjFiMGJhYWEzOmIyZWFjMjViLTAwOTctNGE3YS05Y2FiLWE4MDNhZDA1Mzc3Yw==', '/content/rushifteval1.tsv', '/content/shift_eval_prompt.yaml', 'rushifteval1.csv')

In [15]:
gigachat_function('MTgzMjI0NTktMDA3Ny00Y2NlLTg4MDctMzkwNjFiMGJhYWEzOmIyZWFjMjViLTAwOTctNGE3YS05Y2FiLWE4MDNhZDA1Mzc3Yw==', '/content/rushifteval2.tsv', '/content/shift_eval_prompt.yaml', 'rushifteval2.csv')

In [23]:
gigachat_function('YWVkOTc2NWQtNTEwYi00MDhmLWExM2EtYmYwNDJiNjg2MTRhOjlkZDcyOWVlLTZiODItNDcxNC1hZjUzLTU1ZjMxYWM2MTUxNA==', '/content/rushifteval3.tsv', '/content/shift_eval_prompt.yaml', 'rushifteval3.csv')

[{'авторитет': '2: значения слова отличаются.'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2: значения слова отличаются.'}, {'авторитет': '3'}, {'авторитет': '2'}, {'авторитет': 'Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?'}, {'авторитет': '2: значения слова отличаются.'}, {'авторитет': '2'}, {'авторитет': '3'}, {'авторитет': '2: значения слова отличаются.'}, {'авторитет': '2'}, {'авторитет': '3'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2: значения слова отличаются.'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2: значения слова отличаются.'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2: значения слова отличаются.'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2'}, {'авторитет': '2: значения слова отличаются.'}, {'амбиция': '2'}, {'амбиция': '2'}, {'амбиция': '2'}, {'амбиция': '2'}, {'амбиция': '2'}, {'амбиция': '2'},

In [29]:
df1 = pd.read_csv('/content/rushifteval1.csv', header=None)

In [31]:
df2 = pd.read_csv('/content/rushifteval2.csv', header=None)

In [30]:
df3 = pd.read_csv('/content/rushifteval3.csv', header=None)

In [36]:
final_d = {'word': df1[0].tolist(), '1': df1[1].tolist(), '2': df2[1].tolist(), '3': df3[1].tolist()}

In [37]:
final_df = pd.DataFrame.from_dict(final_d)

In [39]:
final_df

,word,1,2,3
0,авторитет,2.000000,2.052632,2.142857
1,амбиция,2.045455,2.181818,2.086957
2,апостол,1.750000,2.222222,1.875000
3,благодарность,2.166667,2.142857,2.074074
4,блин,2.360000,2.120000,2.080000
5,блондин,2.090909,2.000000,2.130435
6,брат,2.103448,2.120000,2.115385
7,бригада,2.130435,2.090909,2.080000
8,веер,2.086957,2.173913,2.086957
9,век,2.041667,2.095238,2.000000


In [41]:
final_df.to_csv('final_df.csv', header=False, index=False)